In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data.csv")

In [7]:
import pandas as pd

# Step 1: Extract first 8 digits of VIN
vin_prefixes = df['VIN'].str[:8]

vin_prefix_counts = vin_prefixes.value_counts()
print(vin_prefix_counts.describe())  # Stats on counts (mean, min, max, etc.)

count    9353.000000
mean        4.888592
std        12.359977
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       485.000000
Name: count, dtype: float64


In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_predict, LeaveOneOut
from datetime import datetime

def get_price_estimate(vin: str, df: pd.DataFrame, min_samples_for_knn: int = 10, k: int = 5) -> dict:
    """
    Provides a price estimate for a vehicle based on its VIN prefix.

    - If fewer than `min_samples_for_knn` records are found, it provides a
      purely statistical estimate (mean, median).
    - If sufficient records are found, it provides both a statistical estimate
      and a more robust estimate using a K-Nearest Neighbors model. The KNN
      estimate is a smoothed average derived from leave-one-out cross-validation
      on the existing data for that VIN prefix.

    Args:
        vin (str): The 17-digit VIN of the vehicle.
        df (pd.DataFrame): The DataFrame with historical data. Must include:
                           'VIN', 'Sale Price', 'Odometer Reading', 'Lot Year'.
        min_samples_for_knn (int): Min records needed to attempt KNN estimation.
        k (int): The number of neighbors to use for KNN.

    Returns:
        dict: A dictionary containing the status and all generated estimates.
    """
    # --- 1. Input Validation and Preparation ---
    if not isinstance(vin, str) or len(vin) < 8:
        return {'status': 'Error: Invalid VIN provided.', 'estimate': None}
    
    required_cols = ['VIN', 'Sale Price', 'Odometer Reading', 'Lot Year']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"Input DataFrame is missing one or more required columns: {required_cols}")

    vin_prefix = vin[:8]
    if 'VIN_Prefix' not in df.columns:
        df['VIN_Prefix'] = df['VIN'].str[:8]
        
    sub_df = df[df['VIN_Prefix'] == vin_prefix].copy()
    matches_found = len(sub_df)

    # Initialize result
    result = {
        'query_vin_prefix': vin_prefix,
        'matches_found': matches_found,
        'status': '',
        'statistical_estimate': None,
        'knn_estimate': None
    }

    # --- 2. Handle Case: No Data ---
    if matches_found == 0:
        result['status'] = 'Error: No historical data found for this VIN prefix.'
        return result

    # --- 3. Statistical Estimation (always calculated if data exists) ---
    price_data = sub_df['Sale Price']
    stats = {
        'mean_price': round(price_data.mean(), 2),
        'median_price': round(price_data.median(), 2),
        'std_dev_price': round(price_data.std(), 2) if matches_found > 1 else 0.0,
        'min_price': round(price_data.min(), 2),
        'max_price': round(price_data.max(), 2)
    }
    result['statistical_estimate'] = stats
    
    # --- 4. Handle Case: Insufficient Data for KNN ---
    if matches_found < min_samples_for_knn:
        result['status'] = f'Success (Statistical Only): Not enough data for KNN (found {matches_found}, need {min_samples_for_knn}).'
        return result

    # --- 5. Handle Case: Sufficient Data for KNN Estimation ---
    try:
        # a. Feature Engineering for the subgroup
        sub_df['Vehicle Age'] = datetime.now().year - sub_df['Lot Year']
        features = ['Odometer Reading', 'Vehicle Age']
        X_sub = sub_df[features]
        y_sub = sub_df['Sale Price']

        # b. Scale features
        scaler = StandardScaler()
        X_sub_scaled = scaler.fit_transform(X_sub)

        # c. Initialize KNN
        # Ensure k is not larger than the number of available samples minus one
        actual_k = min(k, matches_found - 1)
        if actual_k < 1:
            raise ValueError("k must be at least 1 for KNN.")
            
        knn = KNeighborsRegressor(n_neighbors=actual_k)

        # d. Get robust internal predictions using Leave-One-Out Cross-Validation
        # This is an efficient way to get a smoothed, model-based average
        loocv_predictions = cross_val_predict(knn, X_sub_scaled, y_sub, cv=LeaveOneOut())
        
        # The KNN estimate is the average of these internal predictions
        result['knn_estimate'] = round(np.mean(loocv_predictions), 2)
        result['status'] = f'Success: Both estimates generated (KNN used k={actual_k}).'
        
    except Exception as e:
        # If KNN fails for any reason, we still have the statistical estimate
        result['status'] = f"Success (Statistical Only): KNN estimation failed with error: {e}"

    return result


# --- USAGE EXAMPLE ---

# 1. Create a Sample DataFrame
data = {
    'VIN': ['1ABCDEFGH' + str(i) for i in range(15)] + # Common prefix (15 records)
           ['2BCDEFGHI' + str(i) for i in range(4)],      # Rare prefix (4 records)
    'Lot Year': [2018]*5 + [2019]*5 + [2020]*5 + [2015]*4,
    'Odometer Reading': np.random.randint(20000, 80000, 19),
    'Sale Price': list(np.random.normal(20000, 1500, 5)) + 
                  list(np.random.normal(22000, 1500, 5)) + 
                  list(np.random.normal(24000, 1500, 5)) +
                  list(np.random.normal(12000, 1000, 4))
}
main_df = pd.DataFrame(data)

In [16]:
columns = [
    "Lot Year",
    "Lot Make",
    "Lot Model",
    "Sale Price",
    "Lot Run Condition",
    "Sale Title Type",
    "Damage Type Description",
    "Odometer Reading",
    "Lot Fuel Type"
]
main_df = pd.DataFrame(df)

estimator_1 = get_price_estimate('3N1CN8DV8RL867765',main_df)
print(estimator_1)
print("-" * 40)

estimator_2 = get_price_estimate('KMHLW4AK9NU008872',main_df)
print(estimator_2)
print("-" * 40)

{'query_vin_prefix': '3N1CN8DV', 'matches_found': 15, 'status': 'Success: Both estimates generated (KNN used k=5).', 'statistical_estimate': {'mean_price': np.float64(3111.0), 'median_price': np.float64(3100.0), 'std_dev_price': np.float64(1686.29), 'min_price': np.int64(40), 'max_price': np.int64(6200)}, 'knn_estimate': np.float64(3224.53)}
----------------------------------------
{'query_vin_prefix': 'KMHLW4AK', 'matches_found': 2, 'status': 'Success (Statistical Only): Not enough data for KNN (found 2, need 10).', 'statistical_estimate': {'mean_price': np.float64(7850.0), 'median_price': np.float64(7850.0), 'std_dev_price': np.float64(636.4), 'min_price': np.int64(7400), 'max_price': np.int64(8300)}, 'knn_estimate': None}
----------------------------------------
